In [33]:
import cv2
import numpy as np
import math

In [34]:
def distance(refimg, frameimg, M, N, m,n):
    refimg = np.array(refimg).astype(np.int64)
    frameimg = np.array(frameimg)[m: m + M, n: n + N].astype(np.int64)
    return np.sum((frameimg - refimg)**2)

In [35]:
def check(X,Y,x,y):
    return x>=0 and x<X and y>=0 and y<Y

In [36]:
def drawbox(rgb_frame,M,N,min_i,min_j):
    for i in range(min_i - 1, min_i + M + 1):
        rgb_frame[i][min_j - 1][0] = rgb_frame[i][min_j + N][0] = np.int8(0)
        rgb_frame[i][min_j - 1][1] = rgb_frame[i][min_j + N][1] = np.int8(0)
        rgb_frame[i][min_j - 1][2] = rgb_frame[i][min_j + N][2] = np.int8(255)

    for j in range(min_j - 1, min_j + N + 1):
        rgb_frame[min_i - 1][j][0] = rgb_frame[min_i + M][j][0] = np.int8(0)
        rgb_frame[min_i - 1][j][1] = rgb_frame[min_i + M][j][1] = np.int8(0)
        rgb_frame[min_i - 1][j][2] = rgb_frame[min_i + M][j][2] = np.int8(255)
    return rgb_frame

In [37]:
def exhaustive_search(ref_img, frames, frame_rate, p):
    M,N= ref_img.shape
    I,J = frames[0].shape

    output_frames = []
    prev_x = prev_y = 0
    first = 0
    for frame_img in frames:
        d_min = math.inf
        min_i = min_j = -1
        if first == 0:
            for i in range(I - M):
                for j in range(J - N):
                    d = distance(ref_img, frame_img, M, N, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_x = min_i
            prev_y = min_j
            first+=1
        else:
            for i in range(prev_x - p, prev_x + p + 1):
                for j in range(prev_y - p, prev_y + p + 1):
                    if not check(I-M,J-N,i,j):
                        continue
                    d = distance(ref_img, frame_img, M, N, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_x = min_i
            prev_y = min_j

        rgb_frame = cv2.cvtColor(frame_img, cv2.COLOR_GRAY2BGR)
        rgb_frame = drawbox(rgb_frame,M,N,min_i,min_j)

        output_frames.append(rgb_frame)

    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter('Output_Videos/out_exhaust_' + str(p) + '.avi', fourcc, frame_rate, (J, I), True)

    for output_frame in output_frames:
        out.write(output_frame)
    out.release()

    print("Output video saved for exhaustive search")

In [38]:

def logarithmic_search(ref_img, frames, frame_rate, p):
    M,N = ref_img.shape
    I,J = frames[0].shape

    output_frames = []
    prev_x = prev_y = 0
    first = 0

    for frame_img in frames:
        d_min = math.inf
        min_i = min_j = -1
        if first == 0:
            for i in range(I - M):
                for j in range(J - N):
                    d = distance(ref_img, frame_img, M, N, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_x = min_i
            prev_y = min_j
            first+=1
        else:
            p_tmp = p
            spacing = math.pow(2, math.ceil(math.log(p, 2)) - 1)
            while spacing >= 1:
                d_min = math.inf
                min_i = min_j = -1
                for i in [prev_x - p_tmp, prev_x, prev_x + p_tmp]:
                    for j in [prev_y - p_tmp, prev_y, prev_y + p_tmp]:
                        if not check(I-M,J-N,i,j):
                            continue
                        d = distance(ref_img, frame_img, M, N, i, j)
                        if d < d_min:
                            d_min = d
                            min_i = i
                            min_j = j
                prev_x = min_i
                prev_y = min_j
                p_tmp = round(p_tmp / 2.0)
                spacing /= 2

        rgb_frame = cv2.cvtColor(frame_img, cv2.COLOR_GRAY2BGR)

        rgb_frame = drawbox(rgb_frame,M,N,min_i,min_j)

        output_frames.append(rgb_frame)

    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter('Output_Videos/out_log_' + str(p) + '.avi', fourcc, frame_rate, (J, I), True)

    for output_frame in output_frames:
        out.write(output_frame)
    out.release()

    print("Output video saved for 2D logarithmic search")

In [48]:
def hierarchical_search(ref_img, frames, frame_rate, p):
    M,N = ref_img.shape

    I,J = frames[0].shape

    output_frames = []
    prev_x = prev_y = 0
    first = 0

    for frame_img in frames:
        d_min = math.inf
        min_i = min_j = -1
        if first == 0:
            
            for i in range(I - M):
                for j in range(J - N):
                    d = distance(ref_img, frame_img, M, N, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_y = min_i
            prev_x = min_j
            first+=1
        else:
            lvl_refs = [ref_img]
            lvl_frames = [frame_img]
            lvl_refs.append(cv2.pyrDown(lvl_refs[0]))
            lvl_frames.append(cv2.pyrDown(lvl_frames[0]))
            lvl_refs.append(cv2.pyrDown(lvl_refs[1]))
            lvl_frames.append(cv2.pyrDown(lvl_frames[1]))

            p_tmp = round(p/4.0)
            center_x = math.floor(prev_x/4.0)
            center_y = math.floor(prev_y/4.0)
            for i in range(center_y - p_tmp, center_y + p_tmp + 1):
                for j in range(center_x - p_tmp, center_x + p_tmp + 1):
                    I_,J_ = lvl_frames[2].shape
                    M_,N_ = lvl_refs[2].shape
                    if not check(I_-M_,J_-N_,i,j):
                        continue
                    d = distance(lvl_refs[2], lvl_frames[2], M_,N_, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            y1 = min_i
            x1 = min_j

            d_min = math.inf
            p_tmp = 1
            center_x = 2*x1
            center_y = 2*y1
            for i in range(center_y - p_tmp, center_y + p_tmp + 1):
                for j in range(center_x - p_tmp, center_x + p_tmp + 1):
                    I_,J_ = lvl_frames[1].shape
                    M_,N_ = lvl_refs[1].shape
                    if not check(I_-M_,J_-N_,i,j):
                        continue
                    d = distance(lvl_refs[1], lvl_frames[1], M_,N_, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            y2 = min_i
            x2 = min_j

            d_min = math.inf
            p_tmp = 1
            center_x = 2*x2
            center_y = 2*y2
            for i in range(center_y - p_tmp, center_y + p_tmp + 1):
                for j in range(center_x - p_tmp, center_x + p_tmp + 1):
                    I_,J_ = lvl_frames[0].shape
                    M_,N_ = lvl_refs[0].shape
                    if not check(I_-M_,J_-N_,i,j):
                        continue
                    d = distance(lvl_refs[0], lvl_frames[0], M_,N_, i, j)
                    if d < d_min:
                        d_min = d
                        min_i = i
                        min_j = j
            prev_y = min_i
            prev_x = min_j

        rgb_frame = cv2.cvtColor(frame_img, cv2.COLOR_GRAY2BGR)

        rgb_frame = drawbox(rgb_frame,M,N,min_i,min_j)

        output_frames.append(rgb_frame)

    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter('Output_Videos/out_hierarchy_' + str(p) + '.avi', fourcc, frame_rate, (J, I), True)

    for output_frame in output_frames:
        out.write(output_frame)
    out.release()

    print("Output video saved for hierarchical search")


In [40]:
img = cv2.imread('reference.jpg')
refimg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


In [41]:
cap = cv2.VideoCapture('input.mov')
frames = []
fps = cap.get(cv2.CAP_PROP_FPS)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
cap.release()

In [42]:
exhaustive_search(refimg, frames, fps, 8)

Output video saved for exhaustive search


In [43]:
logarithmic_search(refimg, frames, fps, 8)

Output video saved for 2D logarithmic search


In [49]:
hierarchical_search(refimg, frames, fps, 8)

Output video saved for hierarchical search
